## 1. Setup & Imports

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp
from collections import deque
from pathlib import Path

print("✅ All imports successful")
print(f"TF Version: {tf.__version__}")

## 2. Configuration

In [ ]:
IMG_SIZE = 160
MAX_FRAMES = 16
NUM_CLASSES = 2000
MODEL_PATH = r"D:\Samvad_Setu_final\notebooks\Saved_models\wlasl-final.keras"

# Check if model file exists
model_exists = Path(MODEL_PATH).exists()
print(f"Model file exists: {model_exists}")
if model_exists:
    print(f"Model file size: {Path(MODEL_PATH).stat().st_size / (1024*1024):.2f} MB")

## 3. Test 1: MediaPipe Hand Detection

In [ ]:
# Initialize MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.6,
    min_tracking_confidence=0.6
)

def extract_skeleton(frame):
    """Extract hand skeleton (21 keypoints = 42 values)"""
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    res = hands.process(rgb)

    if not res.multi_hand_landmarks:
        return np.zeros((42,), dtype=np.float32)

    coords = []
    for lm in res.multi_hand_landmarks[0].landmark:
        coords.extend([lm.x, lm.y])

    return np.array(coords, dtype=np.float32)

# Test with a blank frame
test_frame = np.ones((480, 640, 3), dtype=np.uint8) * 255
skeleton = extract_skeleton(test_frame)
print(f"✅ Skeleton extraction works")
print(f"   Skeleton shape: {skeleton.shape}")
print(f"   Non-zero values: {np.count_nonzero(skeleton)}")

## 4. Test 2: Build Simple Model (for testing without weight loading issues)

In [ ]:
# Build a simpler model that won't have weight mismatch issues
def build_test_model():
    """Build a lightweight model for testing"""
    
    # Video branch (using Conv2D instead of MobileNetV2 to avoid weight issues)
    video_in = tf.keras.Input(shape=(MAX_FRAMES, IMG_SIZE, IMG_SIZE, 3), name="video_input")
    x_vid = tf.keras.layers.TimeDistributed(
        tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same')
    )(video_in)
    x_vid = tf.keras.layers.TimeDistributed(
        tf.keras.layers.GlobalAveragePooling2D()
    )(x_vid)  # (None, 16, 32)
    x_vid = tf.keras.layers.GlobalAveragePooling1D()(x_vid)  # (None, 32)
    
    # Skeleton branch
    skel_in = tf.keras.Input(shape=(MAX_FRAMES, 42), name="skeleton_input")
    x_skel = tf.keras.layers.Dense(32, activation='relu')(skel_in)  # (None, 16, 32)
    x_skel = tf.keras.layers.GlobalAveragePooling1D()(x_skel)  # (None, 32)
    
    # Fusion
    x = tf.keras.layers.Concatenate()([x_vid, x_skel])  # (None, 64)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    out = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(x)
    
    return tf.keras.Model(inputs=[video_in, skel_in], outputs=out)

print("⏳ Building test model...")
model = build_test_model()
print("✅ Model built successfully")
model.summary()

## 5. Test 3: Inference with Dummy Data

In [ ]:
# Create dummy test data
dummy_video = np.random.randn(1, MAX_FRAMES, IMG_SIZE, IMG_SIZE, 3).astype(np.float32) / 255.0
dummy_skeleton = np.random.randn(1, MAX_FRAMES, 42).astype(np.float32)

print("⏳ Running inference...")
prediction = model.predict([dummy_video, dummy_skeleton], verbose=0)

print(f"✅ Inference successful")
print(f"   Prediction shape: {prediction.shape}")
print(f"   Top-3 classes: {np.argsort(prediction[0])[-3:][::-1]}")
print(f"   Top-3 confidences: {np.sort(prediction[0])[-3:][::-1]}")

## 6. Test 4: Camera Capture & Processing Pipeline

In [ ]:
# Test camera initialization
print("⏳ Testing camera...")
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

ret, frame = cap.read()
if ret:
    print(f"✅ Camera accessible")
    print(f"   Frame shape: {frame.shape}")
    
    # Test preprocessing
    resized = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
    normalized = resized.astype(np.float32) / 255.0
    print(f"   Preprocessed shape: {normalized.shape}")
    print(f"   Value range: [{normalized.min():.3f}, {normalized.max():.3f}]")
else:
    print("❌ Camera not accessible")

cap.release()
print("✅ Camera released")

## 7. Test 5: Full Pipeline (Video + Skeleton)

In [ ]:
print("⏳ Testing full inference pipeline...")

# Initialize buffers
buf_v = deque(maxlen=MAX_FRAMES)
buf_s = deque(maxlen=MAX_FRAMES)

# Simulate collecting frames
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

frames_collected = 0
for i in range(20):  # Collect 20 frames
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.flip(frame, 1)
    fr = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
    fr = fr.astype(np.float32) / 255.0
    
    buf_v.append(fr)
    buf_s.append(extract_skeleton(frame))
    frames_collected += 1
    
    if len(buf_v) == MAX_FRAMES:
        # Run inference
        video_batch = np.expand_dims(np.array(buf_v), 0)
        skel_batch = np.expand_dims(np.array(buf_s), 0)
        
        pred = model.predict([video_batch, skel_batch], verbose=0)
        
        cls = int(np.argmax(pred))
        conf = float(np.max(pred))
        
        print(f"   Frame {i}: Class={cls}, Confidence={conf:.4f}")

cap.release()
print(f"✅ Pipeline test complete")
print(f"   Frames collected: {frames_collected}")

## Summary
- ✅ All components working
- ✅ MediaPipe hand detection operational  
- ✅ Model inference pipeline functional
- ✅ Camera capture and preprocessing OK

**For production**: Replace the test model with the actual `wlasl-final.keras` after resolving weight shape mismatch